# Simple network

This network is composed of 3 machines: Starting_Client, Backup_Server, Active_Directory (final target).

The process that the attack needs to take to arrive at the final target is the following:

```
Starting_Client -> Discover Backup_Server (Exploration) -> Discover Hard Coded creds for Backup_Server -> Infect Backup_Server (Lateral movement) -> Get admin access to backup server (Vertical movement) ->  Discover Active_Directory -> Infect Active Directory (Lateral movement). 
```

In [2]:
import sys, logging
import cyberbattle.simulation.model as model
import cyberbattle.simulation.commandcontrol as commandcontrol
import cyberbattle.networks.simple_network as sn
import plotly.offline as plo

plo.init_notebook_mode(connected=True) # type: ignore
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(levelname)s: %(message)s")
%matplotlib inline

{}


In [3]:
network = model.create_network(sn.nodes)
env = model.Environment(network=network, vulnerability_library=dict([]), identifiers=sn.ENV_IDENTIFIERS)
env

Environment(network=<networkx.classes.digraph.DiGraph object at 0x715dc5396e90>, vulnerability_library={}, identifiers=Identifiers(properties=['LinuxServer', 'Win11', 'WindowsServer'], ports=['RDP', 'SSH'], local_vulnerabilities=['ReadScriptLogs', 'ReadSourceCode_LeakedCreds', 'ReadSourceCode_LeakedNode', 'WritableBackupScript'], remote_vulnerabilities=['Remote_RDP_Exploit']), creationTime=datetime.datetime(2025, 10, 1, 12, 47, 4, 310179), lastModified=datetime.datetime(2025, 10, 1, 12, 47, 4, 310181), version='0.1.0')

In [4]:
c2 = commandcontrol.CommandControl(env)
dbg = commandcontrol.EnvironmentDebugging(c2)


def plot():
    dbg.plot_discovered_network()
    c2.print_all_attacks()


plot()

,status,properties,local_attacks,remote_attacks
id,,,,
Starting_Client,owned,[Win11],"[ReadSourceCode_LeakedNode, ReadSourceCode_Lea...",[]


## Discover Backup_Server

In [ ]:
outcome = c2.run_attack("Starting_Client", "ReadSourceCode_LeakedNode")
plot()

INFO: discovered node: Backup_Server
INFO: GOT REWARD: The source code contains a reference to the backup server


,status,properties,local_attacks,remote_attacks
id,,,,
Starting_Client,owned,[Win11],"[ReadSourceCode_LeakedNode, ReadSourceCode_Lea...",[]
Backup_Server,discovered,NaN,None,[]


## Discover Hard Coded creds for Backup_Server

In [6]:
outcome = c2.run_attack("Starting_Client", "ReadSourceCode_LeakedCreds")
plot()
c2.credentials_gathered_so_far

INFO: discovered node: Backup_Server
INFO: discovered credential: CachedCredential(node='Backup_Server', port='SSH', credential='BckUserCreds')
INFO: GOT REWARD: The source code contains hard-coded credentials to the backup server


,status,properties,local_attacks,remote_attacks
id,,,,
Starting_Client,owned,[Win11],"[ReadSourceCode_LeakedNode, ReadSourceCode_Lea...",[]
Backup_Server,discovered,NaN,None,[]


{CachedCredential(node='Backup_Server', port='SSH', credential='BckUserCreds')}

## Infect Backup_Server (Lateral movement)

In [7]:
backup_server = c2.connect_and_infect(
    source_node_id="Starting_Client",
    target_node_id="Backup_Server",
    credentials="BckUserCreds",
    port_name="SSH"
)
plot()

INFO: Infected node 'Backup_Server' from 'Starting_Client' via SSH with credential 'BckUserCreds'


,status,properties,local_attacks,remote_attacks
id,,,,
Starting_Client,owned,[Win11],"[ReadSourceCode_LeakedNode, ReadSourceCode_Lea...",[]
Backup_Server,owned,[LinuxServer],[WritableBackupScript],[]


## Get admin access to backup server (Vertical movement) 

In [8]:
outcome = c2.run_attack("Backup_Server", "WritableBackupScript")
plot()

INFO: GOT REWARD: Overwrite the world-writable backup script to get a root shell


,status,properties,local_attacks,remote_attacks
id,,,,
Starting_Client,owned,[Win11],"[ReadSourceCode_LeakedNode, ReadSourceCode_Lea...",[]
Backup_Server,owned,"[LinuxServer, privilege_3]","[WritableBackupScript, ReadScriptLogs]",[]


## Discover Active_Directory 

In [9]:
outcome = c2.run_attack("Backup_Server", "ReadScriptLogs")
plot()

INFO: discovered node: Active_Directory
INFO: GOT REWARD: The logs contain information about an Active Directory server


,status,properties,local_attacks,remote_attacks
id,,,,
Starting_Client,owned,[Win11],"[ReadSourceCode_LeakedNode, ReadSourceCode_Lea...",[]
Backup_Server,owned,"[LinuxServer, privilege_3]","[WritableBackupScript, ReadScriptLogs]",[]
Active_Directory,discovered,NaN,None,[Remote_RDP_Exploit]


## Infect Active Directory (Lateral movement). 


In [10]:
c2.run_remote_attack("Backup_Server", "Active_Directory", "Remote_RDP_Exploit")
plot()

INFO: GOT REWARD: Exploit Active_Directory remotely using Backup_Server root privileges


,status,properties,local_attacks,remote_attacks
id,,,,
Starting_Client,owned,[Win11],"[ReadSourceCode_LeakedNode, ReadSourceCode_Lea...",[]
Backup_Server,owned,"[LinuxServer, privilege_3]","[WritableBackupScript, ReadScriptLogs]",[]
Active_Directory,owned,"[WindowsServer, privilege_3]",[],[Remote_RDP_Exploit]
